In [ ]:
# Import packages
import os
from matplotlib import pyplot as plt
import pandas as pd
import datetime

# Import AuTuMN modules
from autumn.settings import Models, Region
from autumn.settings.folders import OUTPUT_DATA_PATH
from autumn.tools.project import get_project
from autumn.tools import db

from autumn.tools.plots.plotter.base_plotter import COLOR_THEME
from autumn.tools.plots.calibration.plots import get_posterior, get_epi_params
from autumn.dashboards.calibration_results.plots import get_uncertainty_df

In [ ]:

# Specify model details
model = Models.COVID_19
region = Region.SRI_LANKA
dirname = "2022-08-31"

In [ ]:
# get the relevant project and output data
project = get_project(model, region)
project_calib_dir = os.path.join(
    OUTPUT_DATA_PATH, "calibrate", project.model_name, project.region_name
)
calib_path = os.path.join(project_calib_dir, dirname)
# Load tables
mcmc_tables = db.load.load_mcmc_tables(calib_path)
mcmc_params = db.load.load_mcmc_params_tables(calib_path)

In [ ]:
def get_epi_params(mcmc_params, strings_to_ignore=("dispersion_param",)):
    """
    Extract only the epidemiological parameters, ignoring the ones that were only used to tune proposal distributions,
    which end in dispersion_param.
    """
    # for param in mcmc_params[0].loc[:, "name"].unique().tolist():
    #     print(param)
    #     if not any(string in param for string in strings_to_ignore):
    #         return param


    return [
        param
        for param in mcmc_params[0].loc[:, "name"].unique().tolist()
        if not any(string in param for string in strings_to_ignore)
    ]

In [ ]:
mcmc_params[0].head(5)

In [ ]:
mcmc_params[0].columns.tolist()

In [ ]:
def get_new_epi_params(mcmc_params, strings_to_ignore=("dispersion_param", "ifr_multiplier")):
    """
    Extract only the epidemiological parameters, ignoring the ones that were only used to tune proposal distributions,
    which end in dispersion_param.
    """

    return [
        param
        for param in mcmc_params[0].columns.tolist()  ## change this line
        if not any(string in param for string in strings_to_ignore)
    ]



In [ ]:
param_names = get_new_epi_params(mcmc_params, strings_to_ignore=("dispersion_param",))

In [ ]:
def get_new_posterior(mcmc_params, mcmc_tables, param_name, burn_in=0):
    weighted_vals = []
    for param_df, run_df in zip(mcmc_params, mcmc_tables):
        table_df = param_df.merge(run_df, left_index=True,  right_index=True)
        unweighted_vals = table_df[param_name]
        weights = table_df.weight
        for v, w in zip(unweighted_vals, weights):
            weighted_vals += [v] * w


In [ ]:
#get the posteriornew distributions
param_names = get_new_epi_params(mcmc_params, strings_to_ignore=("dispersion_param",))
burn_in = 5000
print(get_new_posterior(mcmc_params, mcmc_tables, "contact_rate", burn_in))
for param_name in param_names:
    if param_name == "voc_emergence.alpha_beta.contact_rate_multiplier":
        vals_df_alpha = get_posterior(mcmc_params, mcmc_tables, param_name, burn_in)[param_name]
    if param_name == "voc_emergence.delta.contact_rate_multiplier":
        vals_df_delta = get_posterior(mcmc_params, mcmc_tables, param_name, burn_in)[param_name]

In [ ]:
#plot the ratio of Delta and Alpha contact rate multiplier
fontsize = 24
fig = plt.figure(figsize=(10,8))
axis = fig.add_subplot(1, 1, 1)

VoC_contact_rate_ratio =vals_df_delta/vals_df_alpha
num_bins = 16
VoC_contact_rate_ratio.hist(bins=num_bins, ax=axis, density=True, color = "blue", ec="blue")
print(VoC_contact_rate_ratio.quantile([.025, .5, .975]))
axis.spines['top'].set_visible(False)
axis.spines['right'].set_visible(False)
axis.set_xlabel("ratio of Delta:Alpha contact rate multiplier", fontsize = fontsize)
axis.set_ylabel("frequency", fontsize = fontsize)
axis.tick_params(axis='both', labelsize=fontsize)
axis.grid(False)
fig.tight_layout